In [ ]:
using JLD2
using Plots
using Statistics




In [ ]:

# ==============================================================================
# 1. Load and Format Data from Compressed File
# ==============================================================================

data_file = joinpath(@__DIR__, "all_MPS_data_compressed.jld2")

if !isfile(data_file)
    error("Data file not found at $data_file")
end

println("Loading data from $data_file...")
loaded_data = load(data_file)
results_db = loaded_data["results_db"]

# Helper function to extract coefficients for a specific sigma
function extract_results_for_sigma(db, target_sigma)
    results = Dict{Int, Vector{Float64}}()
    available_Ns = Int[]
    
    for (key, entry) in db
        (N, sigma) = key
        # Floating point comparison
        if isapprox(sigma, target_sigma, atol=1e-10)
            # stored "spectrum" is eigenvalues (s^2). 
            # We take sqrt to get Schmidt Coefficients (s).
            results[N] = sqrt.(entry["spectrum"])
            push!(available_Ns, N)
        end
    end
    return results, sort(available_Ns)
end

# Extract datasets
results_00, N_vals_00 = extract_results_for_sigma(results_db, 0.0)
results_002, N_vals_002 = extract_results_for_sigma(results_db, 0.002)

println("Loaded $(length(results_00)) systems for σ=0.0")
println("Loaded $(length(results_002)) systems for σ=0.002")
println("Available N range: $(minimum(N_vals_00)) to $(maximum(N_vals_00))")

In [ ]:
# ==============================================================================
# 2. Plotting Functions
# ==============================================================================

"""
1. Replicate "Tail Behavior" Plot (Comparing Two Datasets)
Plots the Schmidt coefficient tails for a list of N values on a 2x4 grid.
"""
function plot_tail_comparison(results_1::Dict, results_2::Dict, N_vals::Vector{Int}; 
                              label1="σ=0.0", label2="σ=0.002",
                              save_plot::Bool=false, output_file::String="tail_plot.png")
    
    # Plot Settings
    p = plot(
        layout = (2, 4),
        size = (1200, 600),
        plot_title = "Schmidt Coefficient Tail Behavior (Log Scale)",
        plot_titlefontsize = 16,
        legend = :topright,
        margin = 5Plots.mm
    )

    for (i, N) in enumerate(N_vals)
        if i > 8 break end # Limit to 8 plots for the 2x4 grid
        
        title_str = "N = $N"
        
        # Get and Sort Coefficients
        v1 = haskey(results_1, N) ? sort(results_1[N], rev=true) : nothing
        v2 = haskey(results_2, N) ? sort(results_2[N], rev=true) : nothing

        # Labels only on the first subplot
        l1 = (i == 1) ? label1 : ""
        l2 = (i == 1) ? label2 : ""

        # Plot Second Dataset (e.g., sigma=0.002) - Orange
        if v2 !== nothing
            mask = v2 .> 0
            y = v2[mask]
            x = (1:length(v2))[mask]
            
            plot!(p, subplot=i, x, y, 
                seriestype=:scatter, markershape=:circle, markersize=3, 
                markerstrokewidth=0, color=:darkorange, alpha=0.8, label=l2)
            plot!(p, subplot=i, x, y, 
                seriestype=:path, color=:darkorange, alpha=0.5, label="")
        end

        # Plot First Dataset (e.g., sigma=0.0) - Purple
        if v1 !== nothing
            mask = v1 .> 0
            y = v1[mask]
            x = (1:length(v1))[mask]
            
            plot!(p, subplot=i, x, y, 
                seriestype=:scatter, markershape=:rect, markersize=3, 
                markerstrokewidth=0, color=:purple, alpha=0.4, label=l1)
            plot!(p, subplot=i, x, y, 
                seriestype=:path, color=:purple, alpha=0.5, label="")
        end

        # Formatting
        plot!(p, subplot=i,
            title = title_str,
            xlabel = "Index",
            ylabel = (i % 4 == 1 ? "Coeffs (log)" : ""), # Only label y-axis on left plots
            yaxis = :log10,
            framestyle = :box
        )
    end

    if save_plot
        savefig(p, output_file)
        println("Comparison plot saved to $output_file")
    end

    return p
end

"""
2. Plot Averaged Schmidt Spectrum for a Single N
"""
function plot_single_N_spectrum(results_dict::Dict, N::Int; color=:blue)
    if !haskey(results_dict, N)
        error("N=$N not found in results dictionary.")
    end

    coeffs = sort(results_dict[N], rev=true)
    
    # Filter strictly positive for log plot
    mask = coeffs .> 0
    y = coeffs[mask]
    x = (1:length(coeffs))[mask]

    p = plot(x, y,
        seriestype = :scatter,
        markershape = :circle,
        markersize = 4,
        markerstrokewidth = 0,
        color = color,
        alpha = 0.8,
        label = "N=$N",
        yaxis = :log10,
        title = "Averaged Schmidt Spectrum (N=$N)",
        xlabel = "Schmidt Index",
        ylabel = "Coefficient Value",
        framestyle = :box,
        grid = true
    )
    plot!(p, x, y, seriestype=:path, color=color, alpha=0.5, label="")
    
    return p
end

"""
3. Plot Averaged Spectrum for Multiple N on a Grid (4x2)
"""
function plot_multi_N_grid(results_dict::Dict; 
                           N_values=[12, 16, 20, 24, 28, 32, 36, 40],
                           color=:forestgreen)
    
    p = plot(
        layout = (2, 4),
        size = (1200, 1000),
        plot_title = "Schmidt Spectra Grid",
        plot_titlefontsize = 16,
        left_margin = 5Plots.mm,
        bottom_margin = 5Plots.mm
    )

    for (i, N) in enumerate(N_values)
        if i > 8 break end
        if !haskey(results_dict, N)
            println("Warning: N=$N not found in data, skipping subplot $i")
            continue
        end

        coeffs = sort(results_dict[N], rev=true)
        mask = coeffs .> 0
        y = coeffs[mask]
        x = (1:length(coeffs))[mask]

        plot!(p, subplot=i, x, y, 
            seriestype=:scatter, markershape=:circle, markersize=3, 
            markerstrokewidth=0, color=color, alpha=0.7, label="N=$N")
        
        plot!(p, subplot=i, x, y, 
            seriestype=:path, color=color, alpha=0.5, label="")

        plot!(p, subplot=i,
            title = "N = $N",
            yaxis = :log10,
            xlabel = "Index",
            ylabel = "Coeffs (log)",
            framestyle = :box
        )
    end

    return p
end

"""
4. Single N Comparison: Plots both spectra on one plot
"""
function plot_single_N_comparison(results_1::Dict, results_2::Dict, N::Int; 
                                  label1="σ=0.0", label2="σ=0.002")
    
    if !haskey(results_1, N) && !haskey(results_2, N)
        error("N=$N found in neither results dictionary.")
    end

    p = plot(
        title = "Schmidt Spectrum Comparison (N=$N)",
        xlabel = "Schmidt Index",
        ylabel = "Coefficient Value",
        yaxis = :log10,
        framestyle = :box,
        grid = true,
        legend = :topright
    )
    
    # Dataset 2 (Orange)
    if haskey(results_2, N)
        v2 = sort(results_2[N], rev=true)
        mask2 = v2 .> 0
        y2 = v2[mask2]
        x2 = (1:length(v2))[mask2]
        plot!(p, x2, y2, seriestype=:scatter, markershape=:circle, markersize=4, 
            markerstrokewidth=0, color=:darkorange, alpha=0.8, label=label2)
        plot!(p, x2, y2, seriestype=:path, color=:darkorange, alpha=0.5, label="")
    end
    
    # Dataset 1 (Purple)
    if haskey(results_1, N)
        v1 = sort(results_1[N], rev=true)
        mask1 = v1 .> 0
        y1 = v1[mask1]
        x1 = (1:length(v1))[mask1]
        plot!(p, x1, y1, seriestype=:scatter, markershape=:rect, markersize=4, 
            markerstrokewidth=0, color=:purple, alpha=0.5, label=label1)
        plot!(p, x1, y1, seriestype=:path, color=:purple, alpha=0.5, label="")
    end
    
    return p
end